In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from tensorflow import keras
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

# Warning
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Random seeds
import random
import numpy as np
import tensorflow as tf
random.seed(319)
np.random.seed(319)
tf.random.set_seed(319)

<a id=0></a>
## <p style="background-color:lightblue; font-family:newtimeroman; font-size:120%; text-align:left; border-radius: 15px 50px;">Table of Content</p>
* [1. Introduction and updates](#1)
* [2. Data Preparation](#2)
    * [2.1. Load Data](#2.1)
    * [2.2. Count record number per each category](#2.2)
    * [2.3. Check null and missing](#2.3)
    * [2.4. Preprocessing](#2.4)
* [3. Model](#3)
    * [3.1 Define Model](#3.1)
    * [3.2 Reducing Learning Rate](#3.2)
    * [3.3 Data Augmentation](#3.3)

* [References](#10)

<a id='1'></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">1. Introduction and updates</p>
 

<a id='2'></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> 2. Data Preparation</p>


<a id=2.1 ></a>
## <p style="background-color:skyblue; font-family:newtimeroman; font-size:140%; text-align:left; border-radius: 20px 50px;">2.1 Load Data</p>

[Content](#0)

In [ ]:
train_full = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
pixels = train_full.drop(labels=["label"],axis=1, inplace=False)
labels =  train_full['label']

In [ ]:
# Free memory
del train_full

<a id=2.2 ></a>
## <p style="background-color:skyblue; font-family:newtimeroman; font-size:140%; text-align:left; border-radius: 20px 50px;">2.2 Count record number per each category</p>

[Content](#0)


In [ ]:
sns.countplot(labels)

### Data is distributed similarity from number 0 to 9 

<a id=2.3 ></a>
## <p style="background-color:skyblue; font-family:newtimeroman; font-size:140%; text-align:left; border-radius: 20px 50px;">2.3. Check null and missing</p>

[Content](#0)


In [ ]:
pixels.isnull().any().describe()

In [ ]:
labels.isnull().any()

In [ ]:
test.isnull().any().describe()

### No corrupted images(missing/null inside)

<a id=2.4 ></a>
## <p style="background-color:skyblue; font-family:newtimeroman; font-size:140%; text-align:left; border-radius: 20px 50px;">2.4. Preprocessing</p>

[Content](#0)


In [ ]:
# Normalization
pixels = pixels/255
test = test/255

In [ ]:
# Reshape
pixels = pixels.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [ ]:
# Label encoding
labels = to_categorical(labels, num_classes=10)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(pixels, labels,
                                                 test_size=0.1,
                                                 random_state=42)

<a id=3 ></a>
## <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 20px 50px;">3. CNN Model</p>

[Content](#0)


<a id=3.1 ></a>
## <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:left; border-radius: 20px 50px;">3.1. Define Model</p>

[Content](#0)


In [ ]:
kernel_initializer = tf.keras.initializers.GlorotNormal(seed=319)

In [ ]:
def create_model():
    model = Sequential()

    model.add(Input(shape=(28,28,1)))
    model.add(Conv2D(32, 
                     kernel_size=(5,5), 
                     kernel_initializer=kernel_initializer,
                     padding="Same", 
                     activation="relu",
                    ))
    model.add(Flatten())
    model.add(Dense(10, 
                    activation="softmax",
                    kernel_initializer=kernel_initializer,
                   ))
    # Define the optimizer
    optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    
    #Compile model
    model.compile(optimizer=optimizer,
              loss="categorical_crossentropy",
              metrics=['accuracy']
             )
    return model

### Create the first model

In [ ]:
model = create_model()
model.summary()

In [ ]:
model.fit(X_train, y_train, 
         epochs=2,
         validation_data=(X_val,y_val))

### Create the second model to compare

In [ ]:
model1 = create_model()
model1.fit(X_train, y_train, 
         epochs=2,
         validation_data=(X_val,y_val))